In [11]:
import json
import random
import openai
import time
import os
openai.api_key = ""
response_return=[]
rationale_1="""
                            You are the grader for a quantum physics class. 
                Your job is to grade the response of the students given a question.
                You will be provided with the question, student response, and ground truth answer.
                You must grade the response base on the criteria (3 points to conclusion (deducted if wrong),4 points to logic and reasoning (minus 2 points for each logic mistake), and 3 points to potential halluciantions (minus 1 for each hallucinated point)) 
                Assign a score within the range of 1 to 10. Generate scoring output in pure json format with two fields, score and rationale.\
                            """
rationale_2="""
                You are the grader for a quantum physics class. 
                Your job is to grade the response of the students given a question.
                You will be provided with the question, student response, and ground truth answer.
                You must grade the response base on the criteria (3 points to conclusion (deducted if wrong),4 points to logic and reasoning (minus 2 points for each logic mistake), and 3 points to potential halluciantions (minus 1 for each hallucinated point)) 
                First answer the question using the ground truth, then use your answer as key to grade. Assign a score within the range of 1 to 10. Generate scoring output in pure json format with three fields, score, rationale, and sample_answer.\
                """
def grade_individual_hints(quiz_data, graded_data_path):
    for item_index, item in enumerate(quiz_data):
        
        # Extract keys and shuffle them without losing the correlation
        hint_keys = ['No_hint', 'Hint1', 'Hint2', 'Hint3']
        random.shuffle(hint_keys)

        # Print out each hint for grading
        for key in hint_keys:
            question = item['question']
            student_response = item[key]
            ground_truth = item['answer']
            response = openai.ChatCompletion.create(
                        model="gpt-4-1106-preview",
                        temperature=0.2,
                        max_tokens=1200,
                        response_format={ "type": "json_object" },
                        messages=[
                            {"role": "system", "content": rationale_2}, 
                            {"role": "assistant", "content": "Treat the following as ground truth: "+ground_truth},
                            {"role": "user", "content": "The question is "+question+". The student response is "+student_response}, 
                    #        {"role": "assistant", "content": 
                    #         """
                    #         Here are some facts, treat these as ground truth when grading the response. {ground_truth}
                    #         """}
                            ],
                    )
            response_str=response.choices[0].message.content
            response_return.append(response_str)
            try:
                # Attempt to parse the JSON content
                parsed_json = json.loads(response_str)
            
                # Extract the relevant information
                grade = parsed_json.get('score')
                rationale = parsed_json.get('rationale')
                sample=parsed_json.get('sample_answer') # remove for rationale_1
                print(grade)
                print(rationale)
                
                # Store the grade and rationale in the item
                item[key] = {'content': item[key], 'grade': grade, 'rationale': rationale, 'sample_a':sample}
            except Exception as e:
                print(f"An error occurred: {e}")
                print(response_str)
            
        # Save the graded item to the file
        save_progress(quiz_data, 'auto_graded_GPT4_2.json')
        time.sleep(60)
        
def save_progress(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4)

def read_json_file(filepath):
    """
    This function reads a JSON file from the given filepath and returns the data.
    If the file is not found, is not a valid JSON, or another error occurs,
    it will handle the exception and return None.

    :param filepath: The path to the JSON file to be read.
    :return: Parsed JSON data or None if an error occurs.
    """
    try:
        with open(filepath, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: The file at {filepath} was not found.")
    except json.JSONDecodeError:
        print(f"Error: The file at {filepath} is not a valid JSON file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return None

# Path to the input and output JSON files
input_json_file_path = 'GPT4_results_wq.json'
output_json_file_path = 'auto_graded_GPT4_2.json'

# Load existing data if available, else load the original quiz hints
graded_quiz_hints = read_json_file(output_json_file_path)
if not graded_quiz_hints:
    graded_quiz_hints = read_json_file(input_json_file_path)

# Shuffle the hints, grade them, and collect the rationales
grade_individual_hints(graded_quiz_hints, output_json_file_path)

print("Grading complete. All items have been saved.")


Error: The file at auto_graded_GPT4_2.json was not found.
7
The student's response is mostly correct in terms of logic and reasoning, but there is a slight misunderstanding in the conclusion. The student correctly states that for the particles measured along the z-axis first, all of them would have been spin-up initially, and after the second measurement along the x-axis, about 50% would be found spin-up along x. This is correct and follows from the principles of quantum mechanics. However, for the particles measured along the x-axis first, the student correctly states that about 50% would be spin-up along x, but then incorrectly concludes that these would all still be found up in the second measurement along x. In reality, after the first measurement along x, the state is collapsed to either spin-up or spin-down along x, and a subsequent measurement along x would still have a 50% chance of being spin-up or spin-down, not 100%. Therefore, the overall fraction of particles that had spin

In [10]:
with open('auto_graded_GPT4_2.json', 'w', encoding='utf-8') as file:
        json.dump(response_return, file, indent=4)